### Test function to infer UTR position from GTF

In [1]:
from utils import load_annotation

In [10]:
import pandas as pd
from joblib import Parallel, delayed
from tqdm.auto import tqdm


def update_utr_features(df: pd.DataFrame, n_jobs: int = 4) -> pd.DataFrame:
    """
    Update UTR features in the GTF DataFrame to either "three_prime_UTR" or "five_prime_UTR"
    based on their relative position with their parent transcript, considering the strand information.

    Args:
        df (pd.DataFrame): Input GTF DataFrame.
        n_jobs (int): Number of parallel jobs to run. Default is 4.

    Returns:
        pd.DataFrame: Updated GTF DataFrame with UTR features renamed.
    """

    def parse_attributes(attribute: str) -> dict:
        """
        Parse GTF attribute string into a dictionary.

        Args:
            attribute (str): Attribute string from GTF file.

        Returns:
            dict: Parsed attributes as a dictionary.
        """
        attr_dict = {}
        for attr in attribute.split(";"):
            key_value = attr.strip().split(" ")
            if len(key_value) == 2:
                key, value = key_value
                value = value.strip('"')
                attr_dict[key] = value
        return attr_dict

    def process_utr(utr_row: pd.Series, transcript_positions: dict) -> pd.Series:
        """
        Process a UTR row to determine if it is a five_prime_UTR or three_prime_UTR.

        Args:
            utr_row (pd.Series): UTR row from the DataFrame.
            transcript_positions (dict): Dictionary containing start and end positions of transcripts.

        Returns:
            pd.Series: Updated UTR row.
        """
        attributes = parse_attributes(utr_row["attribute"])
        transcript_id = attributes.get("transcript_id")
        if not transcript_id or transcript_id not in transcript_positions:
            return utr_row

        start, end = utr_row["start"], utr_row["end"]
        transcript_start, transcript_end, strand = transcript_positions[transcript_id]

        if strand == "+":
            if start == transcript_start:
                utr_row["feature"] = "five_prime_UTR"
            elif end == transcript_end:
                utr_row["feature"] = "three_prime_UTR"
            else:
                utr_row["feature"] = "UTR"
        elif strand == "-":
            if start == transcript_start:
                utr_row["feature"] = "three_prime_UTR"
            elif end == transcript_end:
                utr_row["feature"] = "five_prime_UTR"
            else:
                utr_row["feature"] = "UTR"
        return utr_row

    # Extract transcript positions
    transcripts = df[df["feature"] == "transcript"]
    transcript_positions = {}
    for _, row in transcripts.iterrows():
        attributes = parse_attributes(row["attribute"])
        transcript_id = attributes.get("transcript_id")
        if transcript_id:
            transcript_positions[transcript_id] = (
                row["start"],
                row["end"],
                row["strand"],
            )

    # Process UTR rows
    utr_df = df[df["feature"] == "UTR"]
    non_utr_df = df[df["feature"] != "UTR"]

    # processed_utrs = Parallel(n_jobs=n_jobs)(
    #     delayed(process_utr)(row, transcript_positions)
    #     for _, row in tqdm(list(utr_df.iterrows()), total=len(utr_df))
    # )
    processed_utrs = []
    for _, row in tqdm(list(utr_df.iterrows()), total=len(utr_df)):
        processed_utrs.append(process_utr(row, transcript_positions))

    updated_utr_df = pd.DataFrame(processed_utrs, columns=df.columns)
    updated_df = pd.concat([non_utr_df, updated_utr_df]).sort_index()

    return updated_df


# Example usage
# gtf_df = pd.read_csv('your_gtf_file.gtf', sep='\t', header=None, comment='#',
#                      names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])
# updated_gtf_df = update_utr_features(gtf_df)

In [6]:
df = load_annotation(
    "/mnt/c/aws_data/data/10x/space_ranger/reference/refdata-gex-GRCh38-2020-A/genes/genes.gtf"
)
df.feature.value_counts()

feature
exon              1305354
CDS                754501
UTR                304544
transcript         199138
start_codon         86811
stop_codon          78905
gene                36600
Selenocysteine        115
Name: count, dtype: int64

In [37]:
def parse_attributes(attribute: str) -> dict:
    """
    Parse GTF attribute string into a dictionary.

    Args:
        attribute (str): Attribute string from GTF file.

    Returns:
        dict: Parsed attributes as a dictionary.
    """
    attr_dict = {}
    for attr in attribute.split(";"):
        key_value = attr.strip().split(" ")
        if len(key_value) == 2:
            key, value = key_value
            value = value.strip('"')
            attr_dict[key] = value
    return attr_dict


def process_utr(utr_row: pd.Series, cds_positions: dict) -> pd.Series:
    """
    Process a UTR row to determine if it is a five_prime_UTR or three_prime_UTR.

    Args:
        utr_row (pd.Series): UTR row from the DataFrame.
        transcript_positions (dict): Dictionary containing start and end positions of transcripts.

    Returns:
        pd.Series: Updated UTR row.
    """
    attributes = parse_attributes(utr_row["attribute"])
    protein_id = attributes.get("protein_id")
    if not protein_id or protein_id not in cds_positions:
        return utr_row

    start, end = utr_row["start"], utr_row["end"]
    transcript_start, transcript_end, strand = cds_positions[protein_id]

    feature = "UTR"
    if strand == "+":
        if end < transcript_start:
            feature = "five_prime_UTR"
        elif start > transcript_end:
            feature = "three_prime_UTR"
    elif strand == "-":
        if start < transcript_start:
            feature = "three_prime_UTR"
        elif end > transcript_end:
            feature = "five_prime_UTR"
    return feature

In [27]:
# Extract transcript positions
transcripts = df[df["feature"] == "transcript"]
# transcript_positions = {}
transcript2cds = {}
for _, row in transcripts.iterrows():
    attributes = parse_attributes(row["attribute"])
    transcript_id = attributes.get("transcript_id")
    cds_id = attributes.get("protein_id")
    if transcript_id and cds_id:
        if transcript_id not in transcript2cds:
            transcript2cds[transcript_id] = []
        transcript2cds[transcript_id].append(cds_id)
        # transcript_positions[transcript_id] = (
        #     row["start"],
        #     row["end"],
        #     row["strand"],
        # )
assert max(map(len, transcript2cds.values())) == 1, "Multiple CDS per transcript"

In [29]:
cds_positions = {}
cdss = df[df["feature"] == "CDS"]
for _, row in cdss.iterrows():
    attributes = parse_attributes(row["attribute"])
    cds_id = attributes.get("protein_id")
    if cds_id:
        cds_positions[cds_id] = (row["start"], row["end"], row["strand"])

In [38]:
# Process UTR rows
utr_df = df[df["feature"] == "UTR"]
non_utr_df = df[df["feature"] != "UTR"]

# processed_utrs = []
# for _, row in tqdm(utr_df.iterrows(), total=len(utr_df)):
#     processed_utrs.append(process_utr(row, cds_positions))
features_new = [
    process_utr(row, cds_positions)
    for _, row in tqdm(utr_df.iterrows(), total=len(utr_df))
]
pd.Series(features_new).value_counts()

  0%|          | 0/304544 [00:00<?, ?it/s]

three_prime_UTR    153091
five_prime_UTR     151453
Name: count, dtype: int64

### Extract 3' UTR regions from MACS3 output.

In [1]:
import warnings
import os
import glob

import numpy as np
import pandas as pd
import polars as pl
import pyranges as pr
from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from utils import load_region, load_annotation, read_genomic_region

In [4]:
ref_data_path = "/mnt/c/aws_data/data/ensembl/pub/release-98"
species = "mus_musculus"
annot_path = f"{ref_data_path}/gff3/{species}/Mus_musculus.GRCm38.98.chr_prefix.gff3.gz"

data_path = "/mnt/c/aws_data/data/10x"
exp_name = "visium_fresh_frozen_adult_mouse_brain"
np_path = "/mnt/c/aws_data/data/SpatialSplicing/apa/visium_fresh_frozen_adult_mouse_brain/macs3_out/peaks.narrowPeak"
output_tsv = os.path.splitext(np_path)[0] + ".tsv"

# annot_path = "/mnt/c/aws_data/data/gencode/Gencode_human/release_32/gencode.v32.annotation.gff3.gz"
# sample = "s11"
# np_path = f"/mnt/c/aws_data/data/arora_nc_2022/macs3_out/{sample}_peaks.narrowPeak"
# output_tsv = f"/mnt/c/aws_data/data/arora_nc_2022/macs3_out/{sample}_peaks.tsv"

In [6]:
# for i in range(1, 13):
#     print(f"Processing sample {i}")
#     sample = f"s{i}"
#     np_path = f"/mnt/c/aws_data/data/arora_nc_2022/macs3_out/{sample}_peaks.narrowPeak"
#     output_tsv = f"/mnt/c/aws_data/data/arora_nc_2022/macs3_out/{sample}_peaks.tsv"
for i in range(1):
    feat = "three_prime_UTR" if "gff" in annot_path else "UTR"
    df_peak = read_genomic_region(
        np_path,
        annotation_file=annot_path,
        strand_specific=True,
        feature=feat,
    )
    # aggregate utr_info into list by grouping entries with same chrom, start, end,
    # strand and peak. Drop utr_start, utr_end, utr_strand
    df_peak_g = []
    for name, group in df_peak.groupby(["chrom", "start", "end", "strand", "peak"]):
        df_peak_g.append(
            {
                "chrom": name[0],
                "start": name[1],
                "end": name[2],
                "strand": name[3],
                "peak": name[4],
                "utr_info": ";".join(group[f"{feat}_info"]),
            }
        )
    df_peak_g = pd.DataFrame(df_peak_g)
    df_peak_g
    df_peak_g.index = (
        df_peak_g.chrom
        + "_"
        + df_peak_g.start.astype(str)
        + "_"
        + df_peak_g.end.astype(str)
        + "_"
        + df_peak_g.peak.astype(str)
    )
    print(
        f"Found {df_peak_g.shape[0]} RNA-seq coverage peaks across "
        f"{df_peak[['chrom', 'start', 'end', 'strand']].drop_duplicates().shape[0]} 3' UTR regions"
    )
    # save as tsv
    df_peak_g.to_csv(output_tsv, sep="\t")
    # save as narrowPeak (add dummy columns and remove columns not needed)
    df_peak_g.reset_index(names=["name"]).assign(
        score=999, signalValue=".", pValue=".", qValue="."
    )[
        [
            "chrom",
            "start",
            "end",
            "name",
            "score",
            "strand",
            "signalValue",
            "pValue",
            "qValue",
            "peak",
        ]
    ].sort_values(
        ["chrom", "start", "end", "strand", "peak"]
    ).to_csv(
        f"{os.path.splitext(np_path)[0]}.utr.narrowPeak",
        sep="\t",
        index=False,
        header=False,
    )

    # check whether there are overlapping regions (after dropping duplicates)
    for s in ["+", "-"]:
        chr2arr = {
            c: np.zeros(df_peak_g.query(f"chrom == '{c}'").end.max())
            for c in df_peak_g.chrom.unique()
        }
        for name, row in (
            df_peak_g.query(f"strand == '{s}'")[["chrom", "start", "end", "strand"]]
            .drop_duplicates()
            .iterrows()
        ):
            chrom = row["chrom"]
            start = row["start"]
            end = row["end"]
            chr2arr[chrom][start:end] += 1
        for chr_ in chr2arr:
            max_count = pd.Series(chr2arr[chr_]).value_counts().index.max()
            if max_count > 1:
                print(f"WARNING: {chr_} has overlapping regions")
    # there is no overlapping regions if the maximum count is 1

Empty DataFrame
Columns: [chrom, source, feature, start, end, score, strand, frame, attribute]
Index: []


/tmp/ipykernel_7595/2518696232.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for name, group in df_peak.groupby(["chrom", "start", "end", "strand", "peak"]):


Found 45463 RNA-seq coverage peaks across 32503 3' UTR regions


### Get mapping from transcript id to gene id

In [7]:
from glob import glob

import numpy as np
from scipy import sparse as ss
import pandas as pd
import polars as pl
import scanpy as sc
from tqdm.auto import tqdm, trange

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from utils import load_annotation
from utils_annot import get_gene2name, get_trp2gene

In [10]:
def extract_visium_spot_info_gene_id_count(
    data_dir: str, count_file: str, cells: list[str]
) -> tuple[pd.DataFrame, pd.Series, ss.csr_matrix]:
    """
    Extract cell information from a h5 file.

    Args:
    data_dir (str): Directory containing the h5 file.
    count_file (str): Name of the h5 file.
    cells (list[str]): List of cell names.

    Returns:
    pd.DataFrame: DataFrame containing cell information.
    """
    adata = sc.read_visium(data_dir, count_file=count_file)
    spot_info = adata.obs[["in_tissue", "array_row", "array_col"]]
    spot_info.index = spot_info.index.map(lambda x: x.split("-")[0])
    spot_info = spot_info.loc[cells]
    count = adata.X
    gene_names = adata.var["gene_ids"]
    return spot_info, gene_names, count


def read_spot_info(tissue_position_file: str) -> pd.DataFrame:
    """Read the tissue position file from space ranger output.

    The format of this file changed when space ranger was updated to 2.0, when it added
        header and changed file name from tissue_positions_list.csv to
        tissue_positions.csv. Also for visium HD, the file changed from csv format to
        parquet. The function here deals with all these cases.

    Number of rows in this file is:
        - 4,992 for 6.5mm capture area
        - 14,336 for 11mm capture area
        - 11,222,500 for 2um Visium HD
    """
    if tissue_position_file.endswith(".parquet"):
        spot_info = pd.read_parquet(tissue_position_file)
    else:
        has_header = False
        with open(tissue_position_file, "r") as f:
            first_line = f.readline()
            if first_line.startswith("barcode"):
                has_header = True
        if has_header:
            names = None
        else:
            names = [
                "barcode",
                "in_tissue",
                "array_row",
                "array_col",
                "pxl_row_in_fullres",
                "pxl_col_in_fullres",
            ]
        spot_info = pd.read_csv(tissue_position_file, names=names)
    assert spot_info.shape[0] in [
        4992,
        14336,
        11222500,
    ], f"Unexpected number of rows: {spot_info.shape[0]}"
    return spot_info


# def get_trp2gene(df: pd.DataFrame) -> dict[str, str]:
#     # iter through the attribute column of each row and exact string like
#     # Parent=gene:ENSMUSG00000051951;Name=ENSMUSG00000051951, i.e., look for Parent=gene:xx and Name=xx
#     to_gene = {}
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         parent_gene_id, element_id = None, None
#         for attr in row["attribute"].split(";"):
#             if attr.startswith("Parent=gene:"):
#                 parent_gene_id = attr.split(":")[1]
#             elif attr.startswith("ID=transcript:"):
#                 element_id = attr.split(":")[1]
#         if parent_gene_id and element_id:
#             if parent_gene_id in to_gene:
#                 if not to_gene[element_id] == parent_gene_id:
#                     print(
#                         f"Inconsistent gene names for {element_id}: {to_gene[element_id]} vs {parent_gene_id}"
#                     )
#             else:
#                 to_gene[element_id] = parent_gene_id
#     return to_gene


# def get_gene2name(df: pd.DataFrame) -> dict[str, str]:
#     gene2name = {}
#     for i, row in df.query("feature == 'gene'").iterrows():
#         gene_id, gene_name = None, None
#         for attr in row["attribute"].split(";"):
#             if attr.startswith("ID=gene:"):
#                 gene_id = attr.split(":")[1]
#             elif attr.startswith("Name="):
#                 gene_name = attr.split("=")[1]
#         if gene_id is not None:
#             if gene_id in gene2name:
#                 if not gene2name[gene_id] == gene_name:
#                     print(
#                         f"Inconsistent gene names for {gene_id}: {gene2name[gene_id]} vs {gene_name}"
#                     )
#             else:
#                 gene2name[gene_id] = gene_name
#     return gene2name


def construct_apa_dataset(
    df_count: pl.DataFrame,
    gene2peaks: dict[str, list[str]],
    cells: list[str] | np.ndarray | None,
) -> dict[int, tuple[np.ndarray, pd.DataFrame]]:
    # for each num_peaks, for example num_peaks == 2, construct a np matrix of shape
    # (num_genes_with_this_num_peaks, num_peaks, num_cells), save to npz format
    if cells is None:
        cells = df_count["cell"].unique()
    gene_num_peaks = pd.Series({gene: len(peaks) for gene, peaks in gene2peaks.items()})
    ret = {}
    for num_peaks in gene_num_peaks.unique():
        if num_peaks == 1:
            continue
        num_cells = len(cells)
        genes_with_num_peaks = gene_num_peaks[
            gene_num_peaks == num_peaks
        ].index.to_numpy()
        count = df_count.filter(pl.col("gene").is_in(genes_with_num_peaks))
        count_arrs = []
        for gene in tqdm(genes_with_num_peaks):
            count_g = count.filter(pl.col("gene") == gene)
            peaks = gene2peaks[gene]
            apaxcell = (
                pl.Series(peaks)
                .to_frame("apa_site_name")
                .join(pl.Series(cells).to_frame("cell"), how="cross")
            )
            count_gene_arr = (
                apaxcell.join(count_g, on=["cell", "apa_site_name"], how="left")[
                    "counts"
                ]
                .to_numpy()
                .reshape(num_peaks, num_cells)
            )
            count_arrs.append(count_gene_arr)
        count_arr = np.nan_to_num(np.stack(count_arrs), 0).astype(int)
        ret[num_peaks] = (
            count_arr,
            pd.DataFrame.from_dict(
                {g: gene2peaks[g] for g in genes_with_num_peaks},
                orient="index",
                columns=[f"apa_peak_{i}" for i in range(num_peaks)],
            ),
        )
    return ret

In [11]:
# ref_data_path = "/mnt/c/aws_data/data/ensembl/pub/release-98"
# data_dir_10x = "/mnt/c/aws_data/data/10x"
# apa_data_dir = "/mnt/c/aws_data/data/SpatialSplicing/apa"
# exp_name = "visium_fresh_frozen_adult_mouse_brain"
# species = "mus_musculus"
# annot_path = glob(f"{ref_data_path}/gff3/{species}/*.chr_prefix.gff3.gz")[0]
# peak_path = glob(f"{data_dir_10x}/{exp_name}/*.sorted.macs3_peaks.tsv")[0]
# annot_path = "/mnt/c/aws_data/data/10x/space_ranger/reference/refdata-gex-GRCh38-2020-A/genes/genes.sorted.gtf"
# count_path = f"{apa_data_dir}/{exp_name}/umi_tools_out/counts.tsv.gz"
# npz_path = f"{apa_data_dir}/{exp_name}/spatialsplicing/mtx_apa_{{}}.npz"
# apa_tsv_path = f"{apa_data_dir}/{exp_name}/spatialsplicing/genes_apa_{{}}.tsv"
# spot_info_path = f"{apa_data_dir}/{exp_name}/spatialsplicing/cells.csv"
# gene_ids_save = f"{apa_data_dir}/{exp_name}/spatialsplicing/gene_ids.csv"
# gene_count_save = f"{apa_data_dir}/{exp_name}/spatialsplicing/gene_count.csv"
annot_path = "/mnt/c/aws_data/data/gencode/Gencode_human/release_32/gencode.v32.annotation.gff3.gz"

In [12]:
df = load_annotation(annot_path)
print(df.feature.value_counts())
to_gene = get_trp2gene(df, flavor="gencode-gff3")
gene2name = get_gene2name(df, flavor="gencode-gff3")
len(to_gene), len(gene2name)

feature
exon                                      1372308
CDS                                        762268
transcript                                 227462
three_prime_UTR                            154015
five_prime_UTR                             152793
start_codon                                 87662
stop_codon                                  79913
gene                                        60608
stop_codon_redefined_as_selenocysteine        119
Name: count, dtype: int64


  0%|          | 0/2897148 [00:00<?, ?it/s]

  0%|          | 0/60608 [00:00<?, ?it/s]

(227301, 60563)

In [13]:
sum(g in gene2name for g in to_gene.values()) / len(to_gene)
# those not in the gene2name dict might be lncRNA or other non-coding RNA

0.9999912010945838

In [14]:
if "gff" in annot_path:
    print(df.query("feature == 'three_prime_UTR'").attribute.str.count(";").max())
elif "gtf" in annot_path:
    print(df.query("feature == 'UTR'").attribute.str.count("transcript_id").max())
else:
    raise NotImplementedError

18


In [15]:
for sample_idx in trange(1, 13):
    sample = f"s{sample_idx}"
    peak_path = f"/mnt/c/aws_data/data/arora_nc_2022/macs3_out/{sample}_peaks.tsv"
    count_path = (
        f"/mnt/c/aws_data/data/arora_nc_2022/umi_tools_out/{sample}_counts.tsv.gz"
    )
    spot_info_path = f"/mnt/c/aws_data/data/arora_nc_2022/GSE208253_RAW/{sample}_tissue_positions_list.csv"
    adata_path = f"/mnt/c/aws_data/data/arora_nc_2022/GSE208253_RAW/{sample}_filtered_feature_bc_matrix.h5"
    npz_path = f"/mnt/c/aws_data/data/arora_nc_2022/apa/{sample}_mtx_apa_{{}}.npz"
    apa_tsv_path = f"/mnt/c/aws_data/data/arora_nc_2022/apa/{sample}_genes_apa_{{}}.tsv"
    spot_info_save = f"/mnt/c/aws_data/data/arora_nc_2022/apa/{sample}_cells.csv"
    gene_ids_save = f"/mnt/c/aws_data/data/arora_nc_2022/apa/{sample}_gene_ids.csv"
    gene_count_save = f"/mnt/c/aws_data/data/arora_nc_2022/apa/{sample}_gene_count.csv"
    df_peak = pd.read_table(peak_path, index_col=0)
    print(df_peak.shape)
    if "gff" in annot_path:
        parent_transcript_ids = [
            j.split("=")[1]
            for attr in df_peak.utr_info.str.split(";")
            for j in attr
            if j.startswith("Parent=")
        ]
    elif "gtf" in annot_path:
        parent_transcript_ids = [
            j.split()[1].strip('"')
            for attrs in df_peak.utr_info.str.split(";;")
            for attr in attrs
            for j in attr.split("; ")
            if j.startswith("transcript_id ")
        ]
    else:
        raise NotImplementedError
    print(len(parent_transcript_ids), "transcripts appear in the peak df.")
    print(
        sum([i in to_gene for i in parent_transcript_ids]),
        "of them can be mapped to genes.",
    )
    parent_gene_ids = [to_gene[i] for i in parent_transcript_ids if i in to_gene]
    print(pd.Series(parent_gene_ids).value_counts())

    df_peak["parent_transcripts"] = df_peak.utr_info.apply(
        lambda x: ";".join(
            [attr.split("=")[-1] for attr in x.split(";") if attr.startswith("Parent=")]
        )
    )
    df_peak["parent_genes"] = df_peak.parent_transcripts.apply(
        lambda x: ";".join(list(set(to_gene[i] for i in x.split(";"))))
    )
    peak2genes = df_peak.parent_genes.to_dict()
    peaks_unambiguous = df_peak.query("~parent_genes.str.contains(';')").index.tolist()
    print((df_peak.parent_genes.str.count(";") + 1).value_counts())

    gene2peaks = {}
    # let's filter for those with only one parent gene so that 3' UTR count can be uniquely assigned to a gene
    for peak_name, row in df_peak.query("~parent_genes.str.contains(';')").iterrows():
        gene = row.parent_genes
        if gene in gene2peaks:
            gene2peaks[gene].append(peak_name)
        else:
            gene2peaks[gene] = [peak_name]
    print(pd.Series(gene2peaks).apply(len).value_counts())

    # read peak count and remove ambiguous peaks
    df_count = (
        pl.read_csv(
            count_path,
            separator="\t",
            new_columns=["apa_site_name", "cell", "counts"],
        )
        .filter(pl.col("apa_site_name").is_in(peaks_unambiguous))
        .with_columns(gene=pl.col("apa_site_name").replace(peak2genes))
    )
    cells = df_count["cell"].unique()
    print("Number of cells in APA count table: ", len(cells))

    apa_data = construct_apa_dataset(df_count, gene2peaks, cells)
    for num_peaks, (count_arr, genes_with_num_peaks) in apa_data.items():
        np.savez_compressed(npz_path.format(num_peaks), count_arr)
        genes_with_num_peaks.to_csv(apa_tsv_path.format(num_peaks), sep="\t")

    # spot_info, gene_ids, count = extract_visium_spot_info_gene_id_count(
    #     f"{data_dir_10x}/{exp_name}",
    #     glob(f"{data_dir_10x}/{exp_name}/*raw_feature_bc_matrix.h5")[0],
    #     cells,
    # )

    spot_info = read_spot_info(spot_info_path)
    adata = sc.read_10x_h5(adata_path)
    count = adata.X
    gene_ids = adata.var.gene_ids
    print("adata shape", adata.shape)

    # save visium spot information, a table with gene name and gene id from adata, and count matrix from adata.
    spot_info.to_csv(spot_info_save)
    gene_ids.to_csv(gene_ids_save, header=False)
    ss.save_npz(gene_count_save, count)

  0%|          | 0/12 [00:00<?, ?it/s]

(26183, 6)
56636 transcripts appear in the peak df.
56636 of them can be mapped to genes.
ENSG00000109339.23    129
ENSG00000075711.21    112
ENSG00000198561.13     86
ENSG00000108433.16     81
ENSG00000169255.15     80
                     ... 
ENSG00000169228.14      1
ENSG00000129244.9       1
ENSG00000169223.15      1
ENSG00000169220.18      1
ENSG00000142252.11      1
Name: count, Length: 13602, dtype: int64
parent_genes
1     25429
2       735
3        13
4         2
9         2
15        1
22        1
Name: count, dtype: int64
1     6093
2     3586
3     1929
4      839
5      320
6      134
7       50
8       15
9        7
10       5
11       2
12       1
Name: count, dtype: int64
Number of cells in APA count table:  4907


  0%|          | 0/3586 [00:00<?, ?it/s]

  0%|          | 0/1929 [00:00<?, ?it/s]

  0%|          | 0/839 [00:00<?, ?it/s]

...

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

/home/whatever60/miniforge3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/whatever60/miniforge3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


adata shape (1559, 36601)


In [19]:
count_arr.shape

(5, 9, 4984)

In [80]:
adata = sc.read_visium(
    f"{data_dir_10x}/{exp_name}",
    count_file=glob(f"{data_dir_10x}/{exp_name}/*raw_feature_bc_matrix.h5")[0],
)
adata.X

/home/whatever60/miniforge3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/whatever60/miniforge3/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


<4992x32285 sparse matrix of type '<class 'numpy.float32'>'
	with 13743891 stored elements in Compressed Sparse Row format>

In [83]:
adata.var

,gene_ids,feature_types,genome
Xkr4,ENSMUSG00000051951,Gene Expression,mm10
Gm1992,ENSMUSG00000089699,Gene Expression,mm10
Gm19938,ENSMUSG00000102331,Gene Expression,mm10
Gm37381,ENSMUSG00000102343,Gene Expression,mm10
Rp1,ENSMUSG00000025900,Gene Expression,mm10
...,...,...,...
AC124606.1,ENSMUSG00000095523,Gene Expression,mm10
AC133095.2,ENSMUSG00000095475,Gene Expression,mm10
AC133095.1,ENSMUSG00000094855,Gene Expression,mm10
AC234645.1,ENSMUSG00000095019,Gene Expression,mm10


In [101]:
gff = pd.read_table(
    f"{ref_data_path}/gff3/{species}/Mus_musculus.GRCm38.98.chr_prefix.gff3.gz",
    header=None,
    comment="#",
    names=[
        "chrom",
        "source",
        "feature",
        "start",
        "end",
        "score",
        "strand",
        "frame",
        "attribute",
    ],
)

In [110]:
gff.

0          ID=chromosome:1;Alias=CM000994.2,NC_000067.6,chr1
1                external_name=rank %3D 1;logic_name=firstef
2          external_name=CH29-168G11 (start);logic_name=m...
3          external_name=CH29-168H10 (start);logic_name=m...
4          external_name=DN-332C11 (end);logic_name=mouse...
                                 ...                        
3893614    ID=gene:ENSMUSG00000096850;Name=Gm21748;biotyp...
3893615    ID=transcript:ENSMUST00000179623;Parent=gene:E...
3893616    Parent=transcript:ENSMUST00000179623;Name=ENSM...
3893617    ID=CDS:ENSMUSP00000137361;Parent=transcript:EN...
3893618             external_name=Pseudo;logic_name=trnascan
Name: attribute, Length: 3893619, dtype: object

In [115]:
f"{ref_data_path}/gff3/{species}/Mus_musculus.GRCm38.98.chr_prefix.gff3.gz"

'/mnt/c/aws_data/data/ensembl/pub/release-98/gff3/mus_musculus/Mus_musculus.GRCm38.98.chr_prefix.gff3.gz'

In [114]:
gff.query("feature == 'exon'").attribute.iloc[100]

'Parent=transcript:ENSMUST00000193450;Name=ENSMUSE00001345089;constitutive=0;ensembl_end_phase=-1;ensembl_phase=-1;exon_id=ENSMUSE00001345089;rank=1;version=1'

In [103]:
gff.query("feature == 'three_prime_UTR'")

,chrom,source,feature,start,end,score,strand,frame,attribute
61,chr1,ensembl_havana,three_prime_UTR,3214482,3216021,.,-,.,Parent=transcript:ENSMUST00000070533
659,chr1,ensembl_havana,three_prime_UTR,4344146,4344599,.,-,.,Parent=transcript:ENSMUST00000027032
905,chr1,ensembl_havana,three_prime_UTR,4490931,4491715,.,-,.,Parent=transcript:ENSMUST00000027035
918,chr1,havana,three_prime_UTR,4491250,4491715,.,-,.,Parent=transcript:ENSMUST00000195555
927,chr1,havana,three_prime_UTR,4491390,4491715,.,-,.,Parent=transcript:ENSMUST00000192650
...,...,...,...,...,...,...,...,...,...
3893060,chrY,havana,three_prime_UTR,87575496,87575691,.,+,.,Parent=transcript:ENSMUST00000186493
3893131,chrY,havana,three_prime_UTR,88079298,88079494,.,+,.,Parent=transcript:ENSMUST00000187146
3893274,chrY,havana,three_prime_UTR,89078824,89079015,.,+,.,Parent=transcript:ENSMUST00000186443
3893332,chrY,havana,three_prime_UTR,89713424,89713627,.,-,.,Parent=transcript:ENSMUST00000188269


Code below is for testing purpose

In [36]:
bed_df = load_file(f"{data_path}/{exp_name}/{narrowpeak_path}", "narrowPeak")
annotation_df = load_annotation_file(
    f"{ref_data_path}/gff3/{species}/Mus_musculus.GRCm38.98.chr_prefix.gff3.gz",
    "gff3",
)
bed_df = bed_df.rename(
    columns={
        "chrom": "Chromosome",
        "start": "Start",
        "end": "End",
        "strand": "Strand",
        "name": "Name",
        "score": "Score",
    }
)
annotation_df = annotation_df.rename(
    columns={
        "chrom": "Chromosome",
        "start": "Start",
        "end": "End",
        "strand": "Strand",
        "attribute": "Attribute",
    }
)
bed_pr = pr.PyRanges(bed_df)
utr_pr = pr.PyRanges(annotation_df)

/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:162: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or o

In [95]:
bed_df.iloc[20:].head(10)

,Chromosome,Start,End,Name,Score,Strand,signalValue,pValue,qValue,peak
20,chr1,3648230,3648611,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,432,.,13.38380,45.9135,43.28710,174
21,chr1,3658232,3658599,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,275,.,15.09250,30.0346,27.52980,172
22,chr1,3672580,3672862,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,133,.,7.10059,15.7452,13.39600,136
23,chr1,3676277,3676754,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,414,.,12.87130,44.0919,41.47820,292
24,chr1,3681503,3681836,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,344,.,15.88790,37.0161,34.45460,157
25,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,581,.,13.11190,60.8710,58.14510,186
26,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,779,.,15.90910,80.8293,77.97790,639
27,chr1,4687727,4688068,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,340,.,17.24140,36.6576,34.09880,133
28,chr1,4774305,4774571,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,95,.,3.45395,11.8632,9.57726,141
29,chr1,4776231,4776758,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,1679,.,16.42280,171.6660,167.95600,256


In [129]:
strand_specific = False
overlap_thres = 0.5

In [133]:
# Find overlaps
if strand_specific:
    overlaps = bed_pr.join(utr_pr, strandedness="same", suffix="_utr")
else:
    overlaps = bed_pr.join(utr_pr, strandedness=False, suffix="_utr")

# Calculate overlap lengths and filter by 50% criterion
# overlaps.df["overlap_len"] = overlaps.df.apply(
#     lambda row: min(row.End, row.End_utr) - max(row.Start, row.Start_utr),
#     axis=1,
# )
overlaps = overlaps.insert(
    (
        np.minimum(overlaps.End, overlaps.End_utr)
        - np.maximum(overlaps.Start, overlaps.Start_utr)
    ).rename("overlap_len")
).insert((overlaps.End - overlaps.Start).rename("bed_len"))
overlaps = overlaps[(overlaps.overlap_len / overlaps.bed_len) >= overlap_thres]
overlaps[
    [
        "Chromosome",
        "Start",
        "End",
        "Name",
        "Score",
        "Strand",
        "peak",
        "Start_utr",
        "End_utr",
        "Strand_utr",
        "Attribute",
    ]
].df

/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:162: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  empty_removed = df.groupby("Chromosome")
join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.
/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/init.py:162: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  empty_removed = df.groupby("Chromosome")
/home/whatever60/miniforge3/lib/python3.10/site-packages/pyranges/methods/

,Chromosome,Start,End,Name,Score,Strand,peak,Start_utr,End_utr,Strand_utr,Attribute
0,chr1,3214373,3214776,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,224,.,227,3214482,3216021,-,Parent=transcript:ENSMUST00000070533
1,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,581,.,186,4490931,4491715,-,Parent=transcript:ENSMUST00000027035
2,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,581,.,186,4491250,4491715,-,Parent=transcript:ENSMUST00000195555
3,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,779,.,639,4490931,4491715,-,Parent=transcript:ENSMUST00000027035
4,chr1,4490835,4491694,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,779,.,639,4491250,4491715,-,Parent=transcript:ENSMUST00000195555
...,...,...,...,...,...,...,...,...,...,...,...
39818,chrY,1261434,1261902,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,1148,.,250,1260771,1263287,-,Parent=transcript:ENSMUST00000091190
39819,chrY,1262245,1262560,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,210,.,175,1260771,1263287,-,Parent=transcript:ENSMUST00000091190
39820,chrY,1262245,1262560,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,210,.,175,1262313,1263367,-,Parent=transcript:ENSMUST00000188484
39821,chrY,1262794,1262997,Visium_Fresh_Frozen_Adult_Mouse_Brain_possorte...,109,.,99,1260771,1263287,-,Parent=transcript:ENSMUST00000091190


In [127]:
overlaps[["Chromosome", "Start", "peak"]]

,Chromosome,Start,End,peak
0,chr1,3214373,3214776,227
1,chr1,4490835,4491694,186
2,chr1,4490835,4491694,186
3,chr1,4490835,4491694,639
4,chr1,4490835,4491694,639
...,...,...,...,...
39818,chrY,1261434,1261902,250
39819,chrY,1262245,1262560,175
39820,chrY,1262245,1262560,175
39821,chrY,1262794,1262997,99


In [ ]:
result_df = overlaps.df[
    [
        "Chromosome",
        "Start",
        "End",
        "Name",
        "Score",
        "Strand",
        "Start_utr",
        "End_utr",
        "Strand_utr",
        "attribute",
    ]
]
result_df.columns = [
    "chrom",
    "start",
    "end",
    "name",
    "score",
    "strand",
    "utr_start",
    "utr_end",
    "utr_strand",
    "utr_info",
]
result_df